# V1: SCF optimization with VAMPyR

## Hydrogen atom

We intend to solve the Schrödinger equation for the Hydrogen atom

\begin{equation*}
  \left[\hat{T} + \hat{V}\right]\phi = E\phi
\end{equation*}

where the potential operator contains only the nuclear attraction

\begin{equation*}
  \hat{V}(r) = \frac{-Z}{|R-r|}
\end{equation*}

where $Z=1$ and $R=(0,0,0)$ are the nuclear charge and position, respectively.

### Solution using multiwavelets
In order to solve this equation using multiwavelets we reformulate it in an integral form:

\begin{equation}
  \phi = -2\hat{G}_{\mu}\hat{V}\phi
\end{equation}

and iterate until convergence. $\hat{G}_\mu$ is the Bound-State Helmholtz (BSH) integral operator, whose kernel is defined as

\begin{equation}
  G_\mu(r - r') = \frac{\exp(-\mu |r - r'|)}{4\pi |r - r'|}
\end{equation}

and $\mu=\sqrt{-2E}$ takes the place of the energy in the original Schrödinger equation. For the Hydrogen atom this energy is known (E=-0.5 a.u.) so we can use the "exact" BSH operator when solving for the wavefunction, but in general the energy will have to be computed for each iteration and the BSH operator re-initialized with the updated $\mu$ parameter. For simple systems with a handful of electrons, the straightforward power iteration of the BSH equation is sufficient to achieve convergence, but more complicated molecular systems require additional acceleration techniques (not discussed further).

### Implementation in VAMPyR

What we need for this exercise is:

1. Define a suitable Multi-Resolution Analysis (MRA) for the problem
2. Make analytic nuclear potential and project onto the MRA
3. Make analytic initial guess for the orbital and project onto the MRA
4. Create a Helmholtz operator $\hat{G}$ using the exact value for the Hydrogen atom ($\mu=\sqrt{-2E}$, $E=-0.5 a.u.$)
5. Compute new orbital through application of the Helmholtz operator
$\tilde{\phi}^{n+1} = -2\hat{G}\left[\hat{V}\phi^n\right]$
6. Check for convergence by computing the norm of the orbital update $\Delta\phi^n = \tilde{\phi}^{n+1} - \phi^n$
7. Normalize the orbital $\phi^{n+1} = \tilde{\phi}^{n+1}/||\tilde{\phi}^{n+1}||$
8. Update orbital $\phi^{n} \leftarrow \phi^{n+1}$ for next iteration
9. Repeat steps 5-8 until your orbital has converged

## Solution

Here we put everything together in a single cell for convenience.

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt

# Prepare simple plotting function for visualizations
def line_plot(f, x_min, x_max):
    # Evenly spaced points along the x-axis
    r_x = np.linspace(x_min, x_max, 1000)
    data = [f([x, 0.0, 0.0]) for x in r_x]
    plt.plot(r_x, data)
    return plt

# Analytic nuclear potential
def f_nuc(r):
    Z = 1.0
    R = np.sqrt(r[0]*r[0] + r[1]*r[1] + r[2]*r[2])
    return -Z / R

# Analytic guess for solution
def f_phi(r):
    R2 = r[0]*r[0] + r[1]*r[1] + r[2]*r[2]
    return np.exp(-R2)

# Global parameters
k = 5                        # Polynomial order
L = [-20,20]                 # Simulation box size
epsilon = 1.0e-3             # Relative precision

# Define MRA and multiwavelet projector
MRA = vp.MultiResolutionAnalysis(order=k, box=L)
P_mra = vp.ScalingProjector(mra=MRA, prec=epsilon)

# Project analytic nuclear potential
V_nuc = P_mra(f_nuc)

# Initial guess for the wavefunction
phi_n = P_mra(f_phi)
phi_n.normalize()

# Prepare Helmholtz operator using exact energy
E = -0.5
mu = np.sqrt(-2*E)
G = vp.HelmholtzOperator(mra=MRA, exp=mu, prec=epsilon)
    
# Minimization loop
i = 0
thrs = 1.0e-3
update = 1.0
while (update > thrs):
    # Apply Helmholtz operator
    phi_np1 = -2*G(V_nuc * phi_n)
    
    # Compute norms for convergence check
    norm = phi_np1.norm()
    update = (phi_np1 - phi_n).norm()
    
    # Prepare for next iteration
    phi_n = phi_np1.normalize()
    
    # Plot the current wavefunction
    plt = line_plot(phi_n, -1.0, 1.0)
    
    # Print the current convergence status
    print("iteration: {}    Norm: {}   Update: {}".format(i, norm, update))
    i += 1

# Show the wavefunction plots in the same figure
plt.show()

## V1.2 Extension to Helium

A few things change when you go from Hydrogen to Helium:
1. The energy is no longer known exactly, and thus will have to be computed from the wavefunction
2. The Helmholtz operator which depends on the energy through $\mu = \sqrt{-2E}$ needs to be updated in every iteration
3. The potential operator $V$ depends on the wavefunction and must be updated in every iteration

In this example we will use the Hartree-Fock model, which for a single-orbital system like Helium, reduces to the following potential operator:
\begin{align}
  \hat{V} &= \hat{V}_{nuc} + 2\hat{J} - \hat{K}\\
          &= \hat{V}_{nuc} + \hat{J}
\end{align}
since $\hat{K} = \hat{J}$ for a doubly occupied single orbital.

The Coulomb potential $\hat{J}$ can be computed by application of the Poisson operator $P$:
\begin{equation}
  \hat{J}(r) = P\left[4\pi\rho\right]
\end{equation}
Where $\rho$ is the square of the orbital
\begin{equation}
  \rho = \phi*\phi
\end{equation}

#### Pen and paper exercise:

Use the fact that
\begin{equation}
  \tilde{\phi}^{n+1} = -\Big[\hat{T} - E^n\Big]^{-1} V^n\phi^n \end{equation}
to show that
\begin{equation}
  E^{n+1} = \frac{\langle\tilde{\phi}^{n+1}|\hat{T} +
            \hat{V}^{n+1}|\tilde{\phi}^{n+1}\rangle}
            {||\tilde{\phi}^{n+1}||^2}
\end{equation}
can be written as a pure update $\Delta E^n$ involving only the potentials $\hat{V}^{n+1}$, $\hat{V}^n$ as well as the orbitals $\tilde{\phi}^{n+1}$ and $\phi^n$
\begin{equation}
  E^{n+1} = E^{n} + \Delta E^n
\end{equation}

### Solution

#### Pen and paper exercise

The energy update can be computed with the following expression

\begin{equation}
  \Delta E^n = \frac{\langle{\tilde{\phi}^{n+1}} | V^n | \Delta  \tilde{\phi}^n\rangle + \langle \tilde{\phi}^{n+1} | \Delta V^n | \phi^n \rangle}{\langle \tilde{\phi}^{n+1}|\tilde{\phi}^{n+1} \rangle}
\end{equation}

#### Implementation

#### Implementation exercise:
1. Make a nuclear potential function `f_nuc(r)` for the Helium atom
2. Make an initial guess for the orbital as a python function `f_phi(r)`
3. Project both nuclear potential ($V_{nuc}$) and orbital ($\phi_n$) to the MW basis using a `vp.ScalingProjector` with precision $\epsilon=10^{-3}$
4. Create a Helmholtz operator $G^n$ with $\mu^n = \sqrt{-2E^n}$ using the current energy (start with some reasonable guess)
5. Compute total potential $V^n = V_{nuc} + J^n$, where the Coulomb potential is computed using the `vp.PoissonOperator` on the current squared orbital $J = P\left[4\pi\phi^2\right]$
6. Compute new orbital through application of the Helmholtz operator on $\tilde{\phi}^{n+1} = -2\hat{G}^nV^n\phi^n$
7. Compute the update to the orbital energy ($\Delta E^n$) using the expression from the pen and paper exercise
8. Compute the norm of the orbital update $||\Delta\tilde{\phi}^n|| = ||\tilde{\phi}^{n+1} - \phi^n||$
9. Normalize the orbital $\phi^{n+1} = \tilde{\phi}^{n+1}/||\tilde{\phi}^{n+1}||$
10. Repeat steps 4-9 until your orbital has converged

#### Bonus exercise:
The total energy can be computed after convergence as
$E_{tot} = 2E_n - \langle\rho|J\rangle$, should be around $E_{tot} \approx -2.86$.

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt

# Prepare simple plotting function for visualizations
def line_plot(f, x_min, x_max):
    # Evenly spaced points along the x-axis
    r_x = np.linspace(x_min, x_max, 1000)
    data = [f([x, 0.0, 0.0]) for x in r_x]
    plt.plot(r_x, data)
    return plt

# Analytic nuclear potential
def f_nuc(r):
    Z = 2.0
    R = np.sqrt(r[0]*r[0] + r[1]*r[1] + r[2]*r[2])
    return -Z / R

# Analytic guess for solution
def f_phi(r):
    R2 = r[0]*r[0] + r[1]*r[1] + r[2]*r[2]
    return np.exp(-R2)

# Global parameters
k = 5                        # Polynomial order
L = [-20,20]                 # Simulation box size
epsilon = 1.0e-3             # Relative precision

# Define MRA and multiwavelet projector
MRA = vp.MultiResolutionAnalysis(order=k, box=L)
P_mra = vp.ScalingProjector(mra=MRA, prec=epsilon)

# Project analytic nuclear potential
V_nuc = P_mra(f_nuc)

# Initial guess for the orbital
phi_n = P_mra(f_phi)
phi_n.normalize()

# Initial guess for the orbital energy
E_n = -1.0

# Initialize Poisson operator
P = vp.PoissonOperator(MRA, epsilon)

# Minimization loop
i = 0
thrs = 1.0e-3
update = 1.0
while (update > thrs):
    # Prepare Helmholtz operator using previous energy
    mu_n = np.sqrt(-2*E_n) if E_n < 0.0 else -1.0
    G = vp.HelmholtzOperator(mra=MRA, exp=mu_n, prec=epsilon)
 
    # Compute potential operator
    rho_n = phi_n**2
    J_n = P(4*np.pi*rho_n)
    V_n = V_nuc + J_n
    
    # Apply Helmholtz operator
    phi_np1 = -2*G(V_n * phi_n)
    dPhi_n = phi_np1 - phi_n
    
    # Compute norms for convergence check
    norm = phi_np1.norm()
    update = dPhi_n.norm()
    
    # Compute the potential update
    dRho_n = dPhi_n**2
    dV_n = P(4*np.pi*dRho_n)
    
    # Compute energy update
    dE_n = (vp.dot(phi_np1, V_n * dPhi_n) + vp.dot(phi_np1, dV_n * phi_n)) / phi_np1.squaredNorm()
    
    # Prepare for next iteration
    phi_n = phi_np1.normalize()
    E_n = E_n + dE_n
    
    # Plot the current wavefunction
    plt = line_plot(phi_n, -1.0, 1.0)
    
    # Print the current convergence status
    print("{}  E_n : {}  dE_n : {}  norm: {}  update : {}".format(i, E_n, dE_n, norm, update))
    i += 1

# Compute total energy    
rho = phi_n**2
J = P(4*np.pi*rho)
E_tot = 2*E_n - vp.dot(rho, J)
print("E_tot  : {}".format(E_tot))    
    
# Show the wavefunction plots in the same figure
plt.show()